In [1]:
# 下面开始研究重塑和轴向旋转的问题，用于重新排列表格型数据的基础运算
# 1.重塑层次化索引 
# 层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式。

# 主要功能有：stack 将数据的列旋转为行
# unstack 将数据的行旋转为列

In [2]:
# 合并数据集
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [3]:
data = DataFrame(np.arange(6).reshape((2,3))
                 ,index=pd.Index(['Ohio','Colorado'],name='state')
                 ,columns=pd.Index(['one','two','three'],name='number'))
print(data)

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5


In [4]:
# 将dataframe展开为一个层次化索引的series
result = data.stack()
print(result,type(result))       # 使用该数据的stack方法即可将列转换为行，得到一个Series

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32 <class 'pandas.core.series.Series'>


In [5]:
result.unstack() # 对一个层次化索引的Series，你可以用unstack将其重排为一个DataFrame

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
# 默认情况下，unstack、stack操作的都是最内层，传入分层级别的编号或名称即可对其他级别进行
# unstack操作
print(result)
result.unstack(0)  # 列转行

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [7]:
print(result)
result.unstack('state')   # 行转列 

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [8]:
# 但是，如果不是所有级别值都能在各分组找到的话，则unstack操作可能会引入缺失数据
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
print(data2)
data2.unstack()      # 行转列，默认从最内层解   

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64


,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [9]:
print(data2) 
data2.unstack().stack(dropna=True)  #stack默认是会滤除缺失数据，因此该运算是可逆的,不保留缺失值

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64


one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [10]:
print(data2) 
print(data2.unstack())
data2.unstack().stack(dropna=False)   #  保留缺失值的话，就会出现下面的情况

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64
       a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0


one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [11]:
# 在对dataframe进行unstack操作时，作为旋转轴的级别将会成为结果中的最低级别
print(result)
df = DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))
print(df)

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32
side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10


In [12]:
df.unstack('state')     

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [13]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [14]:
# 将 长格式 旋转为 宽格式
ldata = pd.read_csv('D://work_space_shareit/study/My_python_for_data_analyst/data/macrodata.csv')

ldata.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [15]:
# 下面开始处理该数据集
date = pd.PeriodIndex(year=ldata.year,quarter=ldata.quarter, freq='Q')
print(date)

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')


In [16]:
ldata['date'] = date.asfreq('M', 'e').asfreq('H', 's').values # 利用刚才的periodIndex生成一个新的date列
print(ldata.head())

     year  quarter   realgdp  realcons  realinv  realgovt  realdpi    cpi  \
0  1959.0      1.0  2710.349    1707.4  286.898   470.045   1886.9  28.98   
1  1959.0      2.0  2778.801    1733.7  310.859   481.301   1919.7  29.15   
2  1959.0      3.0  2775.488    1751.8  289.226   491.260   1916.4  29.35   
3  1959.0      4.0  2785.204    1753.7  299.356   484.052   1931.3  29.37   
4  1960.0      1.0  2847.699    1770.5  331.722   462.199   1955.5  29.54   

      m1  tbilrate  unemp      pop  infl  realint              date  
0  139.7      2.82    5.8  177.146  0.00     0.00  1959-03-01 00:00  
1  141.7      3.08    5.1  177.830  2.34     0.74  1959-06-01 00:00  
2  140.5      3.82    5.3  178.657  2.74     1.09  1959-09-01 00:00  
3  140.0      4.33    5.6  179.386  0.27     4.06  1959-12-01 00:00  
4  139.6      3.50    5.2  180.007  2.31     1.19  1960-03-01 00:00  


In [17]:
ldata = ldata.loc[:, ['date', 'realgdp', 'infl', 'unemp']]  # 选取需要的列，其他列去掉
print(ldata.head())

               date   realgdp  infl  unemp
0  1959-03-01 00:00  2710.349  0.00    5.8
1  1959-06-01 00:00  2778.801  2.34    5.1
2  1959-09-01 00:00  2775.488  2.74    5.3
3  1959-12-01 00:00  2785.204  0.27    5.6
4  1960-03-01 00:00  2847.699  2.31    5.2


In [18]:
ldata = pd.melt(ldata, id_vars = ['date'], value_vars=['realgdp', 'infl', 'unemp'], var_name='item')    
# melt与pivot正好对应,用melt将数据融合成需要的亚子
# 将ldata融合为需要的样子的时候也可以选择：ldata.stack(0).unstack(1).head()  的方式
ldata.head()

,date,item,value
0,1959-03-01 00:00,realgdp,2710.349
1,1959-06-01 00:00,realgdp,2778.801
2,1959-09-01 00:00,realgdp,2775.488
3,1959-12-01 00:00,realgdp,2785.204
4,1960-03-01 00:00,realgdp,2847.699


In [19]:
# 将上面的长列格式的数据改为：不同的item值分别形成一列，date列中的时间值则用作索引
# 使用DataFrame中的pivot方法完全可以实现这个转换

In [20]:
pivoted = ldata.pivot('date', 'item', 'value')  # 前两个参数分别用作行和列索引的列名，最后一个参数值则是用于填充DataFrame的数据列的列名。 
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-01 00:00,0.00,2710.349,5.8
1959-06-01 00:00,2.34,2778.801,5.1
1959-09-01 00:00,2.74,2775.488,5.3
1959-12-01 00:00,0.27,2785.204,5.6
1960-03-01 00:00,2.31,2847.699,5.2


In [21]:
# 现假设有两个需要参与重塑的数据列
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-01 00:00,realgdp,2710.349,-0.059049
1,1959-06-01 00:00,realgdp,2778.801,0.523926
2,1959-09-01 00:00,realgdp,2775.488,-0.045128
3,1959-12-01 00:00,realgdp,2785.204,-0.088286
4,1960-03-01 00:00,realgdp,2847.699,0.662183
5,1960-06-01 00:00,realgdp,2834.390,-1.275356
6,1960-09-01 00:00,realgdp,2839.022,-0.016231
7,1960-12-01 00:00,realgdp,2802.616,-0.007782
8,1961-03-01 00:00,realgdp,2819.264,1.526428
9,1961-06-01 00:00,realgdp,2872.005,-1.840416


In [22]:
# 如果要对有两个列参与重塑的dataframe直接进行重塑，忽略最后一个参数的话，得到的DataFrame就会带有层次化的列：
privoted = ldata.pivot('date', 'item')
privoted[:10]

value                    value2                    
item              infl   realgdp unemp      infl   realgdp     unemp
date                                                                
1959-03-01 00:00  0.00  2710.349   5.8  0.780639 -0.059049  1.781558
1959-06-01 00:00  2.34  2778.801   5.1  0.106726  0.523926 -1.482877
1959-09-01 00:00  2.74  2775.488   5.3 -0.273317 -0.045128 -1.327257
1959-12-01 00:00  0.27  2785.204   5.6  0.417315 -0.088286 -0.244335
1960-03-01 00:00  2.31  2847.699   5.2  0.983097  0.662183  0.022557
1960-06-01 00:00  0.14  2834.390   5.2 -0.278714 -1.275356  1.297279
1960-09-01 00:00  2.70  2839.022   5.6  3.153347 -0.016231  0.968079
1960-12-01 00:00  1.21  2802.616   6.3  1.086489 -0.007782 -0.874932
1961-03-01 00:00 -0.40  2819.264   6.8  1.368756  1.526428 -0.725301
1961-06-01 00:00  1.47  2872.005   7.0 -0.565604 -1.840416  0.282447

In [23]:
privoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-01 00:00,0.00,2710.349,5.8
1959-06-01 00:00,2.34,2778.801,5.1
1959-09-01 00:00,2.74,2775.488,5.3
1959-12-01 00:00,0.27,2785.204,5.6
1960-03-01 00:00,2.31,2847.699,5.2


In [25]:
ldata.head()       # 

,date,item,value,value2
0,1959-03-01 00:00,realgdp,2710.349,-0.059049
1,1959-06-01 00:00,realgdp,2778.801,0.523926
2,1959-09-01 00:00,realgdp,2775.488,-0.045128
3,1959-12-01 00:00,realgdp,2785.204,-0.088286
4,1960-03-01 00:00,realgdp,2847.699,0.662183


In [27]:
# 这里需要注意的是：pivot只是一个快捷方式而已，做为替换的话，用set_index创建层次化索引，再用unstack重塑
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2                    
item              infl   realgdp unemp      infl   realgdp     unemp
date                                                                
1959-03-01 00:00  0.00  2710.349   5.8  0.780639 -0.059049  1.781558
1959-06-01 00:00  2.34  2778.801   5.1  0.106726  0.523926 -1.482877
1959-09-01 00:00  2.74  2775.488   5.3 -0.273317 -0.045128 -1.327257
1959-12-01 00:00  0.27  2785.204   5.6  0.417315 -0.088286 -0.244335
1960-03-01 00:00  2.31  2847.699   5.2  0.983097  0.662183  0.022557
1960-06-01 00:00  0.14  2834.390   5.2 -0.278714 -1.275356  1.297279
1960-09-01 00:00  2.70  2839.022   5.6  3.153347 -0.016231  0.968079